In [82]:
import csv
import json
import os
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define file paths using relative paths
base_theme_path = os.path.join('base-theme', 'base-theme.json')
input_folder = 'input'
path_templates_folder = 'path-templates'
output_folder = 'output'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load the base theme file with utf-8-sig encoding to handle BOM
try:
    with open(base_theme_path, 'r', encoding='utf-8-sig') as file:
        base_theme = json.load(file)
    logging.info("Loaded base theme file.")
except Exception as e:
    logging.error(f"Failed to load base theme file: {e}")
    raise

# Function to set value in nested dictionary using dot notation and handling list indices
def set_nested_value(data, path, value):
    keys = re.split(r'\.|\[|\]\.?', path)
    keys = [key for key in keys if key]  # Remove empty strings
    for key in keys[:-1]:
        if key.isdigit():
            key = int(key)
            if isinstance(data, list):
                data = data[key]
            else:
                data = data.setdefault(key, [])
        else:
            data = data.setdefault(key, {})
    last_key = keys[-1]
    if last_key.isdigit():
        last_key = int(last_key)
        if isinstance(data, list):
            data[last_key] = value
        else:
            data[last_key] = value
    else:
        data[last_key] = value

# Function to load variables from a CSV file
def load_variables(variables_path):
    variables = {}
    try:
        with open(variables_path, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                variables[row['Variable Name']] = row['Hex']
        logging.info(f"Loaded variables from {variables_path}.")
    except Exception as e:
        logging.error(f"Failed to load variables from {variables_path}: {e}")
        raise
    return variables

# Function to load paths from a CSV file
def load_paths(paths_path):
    paths = []
    try:
        with open(paths_path, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                paths.append({
                    'Power BI Name': row['Power BI Name'],
                    'Variable': row['Variable'],
                    'Path': row['Path']
                })
        logging.info(f"Loaded paths from {paths_path}.")
    except Exception as e:
        logging.error(f"Failed to load paths from {paths_path}: {e}")
        raise
    return paths

# Function to generate theme files
def generate_theme(variables, paths, output_path):
    # Copy the base theme
    theme = json.loads(json.dumps(base_theme))

    # Update the theme with new colors
    for path_info in paths:
        hex_value = variables.get(path_info['Variable'])
        if hex_value:
            set_nested_value(theme, path_info['Path'], hex_value)

    # Save the new theme file in the output folder
    try:
        with open(output_path, 'w') as file:
            json.dump(theme, file, indent=4)
        logging.info(f"Generated theme file at {output_path}.")
    except Exception as e:
        logging.error(f"Failed to write theme file to {output_path}: {e}")
        raise

# Iterate over all variable files
for variables_file in os.listdir(input_folder):
    if variables_file.startswith('variables_') and variables_file.endswith('.csv'):
        theme_name = variables_file[len('variables_'):-len('.csv')]
        variables_path = os.path.join(input_folder, variables_file)

        # Load variables
        variables = load_variables(variables_path)

        # Iterate over all path templates
        for paths_template_file in os.listdir(path_templates_folder):
            if paths_template_file.endswith('.csv'):
                paths_template_name = paths_template_file[:-len('.csv')]
                paths_path = os.path.join(path_templates_folder, paths_template_file)

                # Load paths
                paths = load_paths(paths_path)

                # Generate the theme
                output_path = os.path.join(output_folder, f'{theme_name}_{paths_template_name}.json')
                generate_theme(variables, paths, output_path)

logging.info("Theme files generation completed.")

2024-05-24 16:02:39,501 - INFO - Loaded base theme file.
2024-05-24 16:02:39,503 - INFO - Loaded variables from input/variables_neutralpop.csv.
2024-05-24 16:02:39,504 - INFO - Loaded paths from path-templates/default.csv.
2024-05-24 16:02:39,507 - INFO - Generated theme file at output/neutralpop_default.json.
2024-05-24 16:02:39,508 - INFO - Loaded paths from path-templates/alternative.csv.
2024-05-24 16:02:39,510 - INFO - Generated theme file at output/neutralpop_alternative.json.
2024-05-24 16:02:39,511 - INFO - Loaded variables from input/variables_sepia.csv.
2024-05-24 16:02:39,511 - INFO - Loaded paths from path-templates/default.csv.
2024-05-24 16:02:39,514 - INFO - Generated theme file at output/sepia_default.json.
2024-05-24 16:02:39,514 - INFO - Loaded paths from path-templates/alternative.csv.
2024-05-24 16:02:39,516 - INFO - Generated theme file at output/sepia_alternative.json.
2024-05-24 16:02:39,517 - INFO - Loaded variables from input/variables_dark.csv.
2024-05-24 16:0